In [1]:
import pandas as pd
import numpy as np
from xgboost import XGBRegressor , XGBRFRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV,train_test_split
from pytorch_tabnet.tab_model import TabNetClassifier, TabNetRegressor

C:\Users\DELL\Miniconda3\lib\site-packages\dask\dataframe\utils.py:13: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [13]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
traintest = train.append(test)
print( 'combined shape',traintest.shape)

y = traintest['Attrition_rate']
traintest.drop(['Attrition_rate','Employee_ID'],axis=1,inplace=True)


traintest['VAR4'].fillna(4.0,inplace=True)
traintest['Age'].fillna(traintest.groupby(['Gender', 'Time_since_promotion'])['Age'].transform('median'),inplace=True)
traintest['VAR2'].fillna(0.0,inplace=True)
traintest['Pay_Scale'].fillna(traintest.Pay_Scale.median(),inplace=True)
traintest['Time_of_service'].fillna(traintest.Time_of_service.mean(),inplace=True)
traintest['Work_Life_balance'].fillna(traintest.Work_Life_balance.mean(),inplace=True)
print('anynull_value',traintest.isnull().any().sum())

combined shape (10000, 24)
anynull_value 0


In [5]:
X['Age'].fillna(X['Age'].mean(),inplace=True)
X['Pay_Scale'].fillna(X.Pay_Scale.median(),inplace=True)
X['Time_of_service'].fillna(X.Time_of_service.mean(),inplace=True)
X['VAR2'].fillna(X.VAR2.median(),inplace=True)
X['VAR4'].fillna(X.VAR4.mode()[0],inplace=True)
X.Work_Life_balance.fillna(X.Work_Life_balance.mean(),inplace=True)
traintest.isnull().any().sum()

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [14]:
#try with one hot encoder
#cat_features =  list(traintest.select_dtypes(include=object).columns)
#traintest = pd.get_dummies(traintest,columns=cat_features,drop_first=True)
#print('after onehot',traintest.shape)

cat_features =  list(traintest.select_dtypes(include=object).columns)
categorical_columns = []
categorical_dims =  {}

for i in cat_features:
    l = LabelEncoder()
    traintest[i] = l.fit_transform(traintest[i].values)
    categorical_columns.append(i)
    categorical_dims[i]=len(l.classes_)
print('after label encoder',traintest.shape)


sc = StandardScaler()
X = sc.fit_transform(traintest)

X_train = traintest.iloc[:7000,:]
Y_train = y[:7000].values.reshape(-1,1)
X_test = traintest.iloc[7000:,:]

x_train , x_val ,y_train , y_val = train_test_split(X_train,Y_train,
                                                      test_size=0.2,random_state=42,shuffle=True)
                       
print('x_train',x_train.shape)
print('x_val',x_val.shape)
print('y_train',y_train.shape)
print('y_val',y_val.shape)

after label encoder (10000, 22)
x_train (5600, 22)
x_val (1400, 22)
y_train (5600, 1)
y_val (1400, 1)


In [9]:
train.columns

Index(['Employee_ID', 'Gender', 'Age', 'Education_Level',
       'Relationship_Status', 'Hometown', 'Unit', 'Decision_skill_possess',
       'Time_of_service', 'Time_since_promotion', 'growth_rate', 'Travel_Rate',
       'Post_Level', 'Pay_Scale', 'Compensation_and_Benefits',
       'Work_Life_balance', 'VAR1', 'VAR2', 'VAR3', 'VAR4', 'VAR5', 'VAR6',
       'VAR7', 'Attrition_rate'],
      dtype='object')

In [15]:
unused_feat=['Employee_ID']
target = ['Attrition_rate']

features = [ col for col in train.columns if col not in unused_feat+[target]] 

cat_idxs = [ i for i, f in enumerate(features) if f in categorical_columns]

cat_dims = [ categorical_dims[f] for i, f in enumerate(features) if f in categorical_columns]

cat_emb_dim = [5, 4, 3, 6, 2, 2]


In [16]:
cat_dims

[2, 2, 5, 12, 4, 5]

In [17]:
categorical_columns

['Gender',
 'Relationship_Status',
 'Hometown',
 'Unit',
 'Decision_skill_possess',
 'Compensation_and_Benefits']

In [18]:
cat_idxs

[0, 3, 4, 5, 6, 13]

In [19]:
params ={
    'n_d':[8,10,16,32],
    'n_a':[8,10,16,32],
    'n_steps':[3,4,5,7],
    'gamma':[1,1.3,1.4,1.8]
    ''
}

SyntaxError: invalid syntax (<ipython-input-19-1c564a536a16>, line 5)

In [21]:
clf = TabNetRegressor(cat_idxs=cat_idxs,cat_dims=cat_dims,
                      verbose=1)
clf.fit(x_train,y_train,x_val,y_val)

Device used : cpu
Will train until validation stopping metric hasn't improved in 10 rounds.
---------------------------------------
| EPOCH |  train  |   valid  | total time (s)


KeyError: 191

In [35]:
ypr=clf.predict(x_val)
print(100*max(0,1-np.sqrt(mean_squared_error(y_val,ypr))))

81.13947656766054


In [36]:
ytabnet = clf.predict(X_test)
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = ytabnet
sub.to_csv('tabnet_submission.csv',index=False)

In [26]:
test.head()

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Pay_Scale,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7
0,EID_22713,F,32.0,5,Single,Springfield,R&D,Conceptual,7.0,4,...,4.0,type2,1.0,3,-0.9612,-0.4537,2.0,1,8,4
1,EID_9658,M,65.0,2,Single,Lebanon,IT,Directive,41.0,2,...,1.0,type2,1.0,4,-0.9612,0.7075,1.0,2,8,2
2,EID_22203,M,52.0,3,Married,Springfield,Sales,Directive,21.0,3,...,8.0,type3,1.0,4,-0.1048,0.7075,2.0,1,9,3
3,EID_7652,M,50.0,5,Single,Washington,Marketing,Analytical,11.0,4,...,2.0,type0,4.0,3,-0.1048,0.7075,2.0,2,8,3
4,EID_6516,F,44.0,3,Married,Franklin,R&D,Conceptual,12.0,4,...,2.0,type2,4.0,4,1.6081,0.7075,2.0,2,7,4


In [39]:
clf_xgb = XGBRegressor(max_depth=8,
    learning_rate=0.1,
    n_estimators=1000,
    verbosity=0,
    silent=None,
    objective='reg:linear',
    booster='gbtree',
    n_jobs=-1,
    nthread=None,
    gamma=0,
    min_child_weight=1,
    max_delta_step=0,
    subsample=0.7,
    colsample_bytree=1,
    colsample_bylevel=1,
    colsample_bynode=1,
    reg_alpha=0,
    reg_lambda=1,
    scale_pos_weight=1,
    base_score=0.5,
    random_state=0,
    seed=None)
clf_xgb.fit(x_train, y_train,
        eval_set=[(x_val, y_val)],
        early_stopping_rounds=40,
        verbose=10)

[0]	validation_0-rmse:0.33727
Will train until validation_0-rmse hasn't improved in 40 rounds.
[10]	validation_0-rmse:0.21476
[20]	validation_0-rmse:0.19335
[30]	validation_0-rmse:0.19043
[40]	validation_0-rmse:0.19146
[50]	validation_0-rmse:0.19219
[60]	validation_0-rmse:0.19298
[70]	validation_0-rmse:0.19342
Stopping. Best iteration:
[30]	validation_0-rmse:0.19043



XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.1, max_delta_step=0, max_depth=8,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=1000, n_jobs=-1, nthread=-1, num_parallel_tree=1,
             objective='reg:linear', random_state=0, reg_alpha=0, reg_lambda=1,
             scale_pos_weight=1, seed=0, silent=None, subsample=0.7,
             tree_method='exact', validate_parameters=1, verbosity=0)

In [41]:
yxgb = np.array(clf_xgb.predict(x_val))
print(100*max(0,1-np.sqrt(mean_squared_error(y_val,yxgb))))



80.95683619328904


In [42]:
pr = np.array(clf_xgb.predict(X_test))
sub = pd.DataFrame()
sub['Employee_ID'] = test['Employee_ID']
sub['Attrition_rate'] = pr
sub.to_csv('xgb_submission.csv',index=False)

In [25]:
train.head(27)

,Employee_ID,Gender,Age,Education_Level,Relationship_Status,Hometown,Unit,Decision_skill_possess,Time_of_service,Time_since_promotion,...,Compensation_and_Benefits,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
0,EID_23371,F,42.0,4,Married,Franklin,IT,Conceptual,4.0,4,...,type2,3.0,4,0.7516,1.8688,2.0,4,5,3,0.1841
1,EID_18000,M,24.0,3,Single,Springfield,Logistics,Analytical,5.0,4,...,type2,4.0,3,-0.9612,-0.4537,2.0,3,5,3,0.0670
2,EID_3891,F,58.0,3,Married,Clinton,Quality,Conceptual,27.0,3,...,type2,1.0,4,-0.9612,-0.4537,3.0,3,8,3,0.0851
3,EID_17492,F,26.0,3,Single,Lebanon,Human Resource Management,Behavioral,4.0,3,...,type2,1.0,3,-1.8176,-0.4537,NaN,3,7,3,0.0668
4,EID_22534,F,31.0,1,Married,Springfield,Logistics,Conceptual,5.0,4,...,type3,3.0,1,0.7516,-0.4537,2.0,2,8,2,0.1827
5,EID_2278,M,54.0,3,Married,Lebanon,Purchasing,Conceptual,19.0,1,...,type2,1.0,3,-1.8176,1.8688,2.0,2,8,3,0.7613
6,EID_18588,F,21.0,4,Married,Springfield,Purchasing,Directive,2.0,1,...,type2,2.0,3,-0.9612,0.7075,2.0,3,7,3,0.2819
7,EID_1235,F,NaN,3,Married,Springfield,Sales,Directive,34.0,4,...,type3,2.0,3,-0.1048,-0.4537,2.0,3,9,3,0.1169
8,EID_10197,M,40.0,4,Single,Springfield,Production,Analytical,13.0,1,...,type0,4.0,1,NaN,1.8688,2.0,5,6,3,0.1968
9,EID_21262,M,45.0,3,Married,Lebanon,IT,Directive,21.0,4,...,type3,4.0,3,0.7516,-0.4537,2.0,4,8,3,0.2870


In [23]:
train.describe()

,Age,Education_Level,Time_of_service,Time_since_promotion,growth_rate,Travel_Rate,Post_Level,Pay_Scale,Work_Life_balance,VAR1,VAR2,VAR3,VAR4,VAR5,VAR6,VAR7,Attrition_rate
count,6588.000000,7000.000000,6856.000000,7000.000000,7000.000000,7000.000000,7000.000000,6991.000000,6989.000000,7000.000000,6423.000000,7000.000000,6344.000000,7000.000000,7000.000000,7000.000000,7000.000000
mean,39.622799,3.187857,13.385064,2.367143,47.064286,0.817857,2.798000,6.006294,2.387895,3.098571,-0.008126,-0.013606,1.891078,2.834143,7.101286,3.257000,0.189376
std,13.606920,1.065102,10.364188,1.149395,15.761406,0.648205,1.163721,2.058435,1.122786,0.836377,0.989850,0.986933,0.529403,0.938945,1.164262,0.925319,0.185753
min,19.000000,1.000000,0.000000,0.000000,20.000000,0.000000,1.000000,1.000000,1.000000,1.000000,-1.817600,-2.776200,1.000000,1.000000,5.000000,1.000000,0.000000
25%,27.000000,3.000000,5.000000,1.000000,33.000000,0.000000,2.000000,5.000000,1.000000,3.000000,-0.961200,-0.453700,2.000000,2.000000,6.000000,3.000000,0.070400
50%,37.000000,3.000000,10.000000,2.000000,47.000000,1.000000,3.000000,6.000000,2.000000,3.000000,-0.104800,-0.453700,2.000000,3.000000,7.000000,3.000000,0.142650
75%,52.000000,4.000000,21.000000,3.000000,61.000000,1.000000,3.000000,8.000000,3.000000,3.000000,0.751600,0.707500,2.000000,3.000000,8.000000,4.000000,0.235000
max,65.000000,5.000000,43.000000,4.000000,74.000000,2.000000,5.000000,10.000000,5.000000,5.000000,1.608100,1.868800,3.000000,5.000000,9.000000,5.000000,0.995900
